In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from numpy import character

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import pandas as pd

books=pd.read_csv("books_cleaned.csv")

In [6]:
books


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883A NOVEL THAT READERS and critics ...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982A new 'Christie for Christmas' --...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736A memorable, mesmerizing heroine ..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897Lewis' work on the nature of love...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934""In The Problem of Pain, C.S. Lew..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222On A Train Journey Home To North ...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014This book tells the tale of a man...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623Wisdom to Create a Life of Passio...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535This collection of the timeless t...


In [7]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep="\n", index=False, header=False)


In [8]:
raw_documents = TextLoader("tagged_description.txt", encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator='\n')
documents = text_splitter.split_documents(raw_documents)




Created a chunk of size 1167, which is longer than the specified 0
Created a chunk of size 1213, which is longer than the specified 0
Created a chunk of size 372, which is longer than the specified 0
Created a chunk of size 308, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 481, which is longer than the specified 0
Created a chunk of size 959, which is longer than the specified 0
Created a chunk of size 187, which is longer than the specified 0
Created a chunk of size 842, which is longer than the specified 0
Created a chunk of size 295, which is longer than the specified 0
Created a chunk of size 196, which is longer than the specified 0
Created a chunk of size 880, which is longer than the specified 0
Created a chunk of size 1087, which is longer than the specified 0
Created a chunk of size 1188, which is longer than the specified 0
Created a chunk of size 303, which is longer than the specified 0
Create

In [9]:
documents[0]


Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gil

In [ ]:
db_books=Chroma.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

In [1]:
query="A book to teach children about nature"
docs=db_books.similarity_search(query, k=10)
docs


NameError: name 'db_books' is not defined

In [ ]:
books[books["isbn13"]==int(docs[0].page_content.spli()[0].strip())]

In [ ]:
def retrieve_semantic_recommendations(
        query:str, top_k: int=10

) -> pd.DataFrame:
    recs=db_books.similarity_search(query, k=50)
    books_list=[]
    for i in range(0,len(recs)):
        books_list +=[int(recs[i]. page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)


In [ ]:
retrieve_semantic_recommendations("A book to teach children about nature")